### Ejercicio 3
Escribir un programa que realice la suma paralela de los N primeros números naturales. Los argumentos del programa fijarán: 
- Primer argumento, el número de threads (Nt). 
- Segundo argumento, el tamaño de bloque (Tb) que determina cuántos números sumará cada thread. 

Cada thread se le asignará un identificador (i = 0,1,2…) y sumará los enteros en el rango [Tb⋅i - (i+1)⋅Tb - 1] que agregará en una variable compartida, suma. El thread principal sincronizará todos los threads y mostrará la suma. 

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

typedef struct _thread_info_t{
	int num_thread;	// Numero de hilo.
	int tb;		// Tamaño de bloque.

	pthread_t id;	// tipo del hilo (su id)
}thread_info_t;

typedef struct _suma_p_t{
	pthread_mutex_t mutex;	// Mutex para la región critica
	int suma;		// Variable compartida
}suma_p_t;

suma_p_t suma = {PTHREAD_MUTEX_INITIALIZER, 0};

void *create_thread(void * arg){
	thread_info_t *info = (thread_info_t *) arg;
	// sumamos
	int suma_parcial = 0;
	int ini = info->tb * info->num_thread;
	int fin = info->tb * (info->num_thread +1);
	for(int i = ini; i < fin; i++){
		suma_parcial += i;
	}
	// Entramos en region critica.
	pthread_mutex_lock(&suma.mutex);
	suma.suma += suma_parcial;
	pthread_mutex_unlock(&suma.mutex);
	printf("Acaba hilo %i\n", info->num_thread);
}

int main(int argc, char *argv[]){
	int n_threads = atoi(argv[1]);
	int t_bloq = atoi(argv[2]);

	// pool de threads, hay que reservar memoria
	thread_info_t *pool = malloc(n_threads * sizeof(thread_info_t));
	// Creamos los threads
	for(int i = 0; i<n_threads;i++){
		pool[i].num_thread = i;	// asignamos el numero de hilo
		pool[i].tb = t_bloq;	// asignamos el tamaño de bloque
		int res = pthread_create(&pool[i].id,		// id del hilo (pthread_t)
					NULL,			// atributos de la funcion
					create_thread,		// funcion a lanzar
					(void *) &pool[i]);	// atributos de la funcion
	}
	// Sincronizamos los hilos
	for(int i = 0; i<n_threads; i++){
		pthread_join(pool[i].id, NULL);
	}
	int suma_real = ((n_threads * t_bloq)* ((n_threads *t_bloq)-1))/2;
	printf("SUMA TOTAL %i/%i\n", suma.suma, suma_real);
	free(pool);
	return 0;
}



### Ejercicio 4